In [ ]:
import datetime

import pickle
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from sklearn import cluster, covariance, manifold

http://scikit-learn.org/stable/auto_examples/applications/plot_stock_market.html

In [ ]:
import sys
import sys
if sys.version_info[0] < 3:
    names, ts_data = pickle.load(open('data/correlation_series_cpu.pickle'))
else:
    names, ts_data = pickle.load(open('data/correlation_series_cpu.pickle', 'rb'), encoding='latin1')

In [ ]:
for name in sorted(names):
    print(name)

In [ ]:
plt.plot(ts_data.T)
plt.show()

In [ ]:
edge_model = covariance.GraphLassoCV()
# edge_model = covariance.EllipticEnvelope()

# standardize the time series: using correlations rather than covariance
# is more efficient for structure recovery
X = ts_data.copy().T
X -= X.mean(axis=0)
X /= X.std(axis=0)

In [ ]:
plt.plot(X)
plt.show()

In [ ]:
edge_model.fit(X)

In [ ]:
edge_model.covariance_

In [ ]:
plt.pcolor(edge_model.covariance_)
plt.colorbar()
plt.show()

In [ ]:
tweaked = np.copy(edge_model.covariance_)
np.fill_diagonal(tweaked, 0)
plt.pcolor(tweaked)
plt.colorbar()
plt.show()

In [ ]:
_, labels = cluster.affinity_propagation(edge_model.covariance_)

In [ ]:
labels.max()

In [ ]:
len(names)

In [ ]:
n_labels = labels.max()

for i in range(n_labels + 1):
    print('Cluster %i: %s' % ((i + 1), ', '.join(names[labels == i])))

### Real node layout
- host001
    - dns001
- host006
    - dns004
- host007
    - cassandracl005
    - elasticsearch004
- host008
    - rumdatareceiver002
- host009
- host010
    - cassandracl006
    - elasticsearch005
- host011
    - cassandracl007
    - elasticsearch006
- host012
    - cassandracl008
    - elasticsearch007
- host013
    - cassandracl002
    - elasticsearch013
- host014
    - cassandracl003
    - elasticsearch009
- host015
    - cassandracl011

In [ ]:
# We use a dense eigen_solver to achieve reproducibility (arpack is
# initiated with random vectors that we don't control). In addition, we
# use a large number of neighbors to capture the large-scale structure.
node_position_model = manifold.LocallyLinearEmbedding(
    n_components=2, eigen_solver='dense', n_neighbors=6)

embedding = node_position_model.fit_transform(X.T).T

In [ ]:
embedding.T

In [ ]:
plt.figure(1, facecolor='w', figsize=(10, 8))
plt.clf()
ax = plt.axes([0., 0., 1., 1.])
plt.axis('off')

# Display a graph of the partial correlations
partial_correlations = edge_model.precision_.copy()
d = 1 / np.sqrt(np.diag(partial_correlations))
partial_correlations *= d
partial_correlations *= d[:, np.newaxis]
non_zero = (np.abs(np.triu(partial_correlations, k=1)) > 0.02)

# Plot the nodes using the coordinates of our embedding
plt.scatter(embedding[0], embedding[1], s=100 * d ** 2, c=labels,
            cmap=plt.cm.spectral)

# Plot the edges
start_idx, end_idx = np.where(non_zero)
#a sequence of (*line0*, *line1*, *line2*), where::
#            linen = (x0, y0), (x1, y1), ... (xm, ym)
segments = [[embedding[:, start], embedding[:, stop]]
            for start, stop in zip(start_idx, end_idx)]
values = np.abs(partial_correlations[non_zero])
lc = LineCollection(segments,
                    zorder=0, cmap=plt.cm.hot_r,
                    norm=plt.Normalize(0, .7 * values.max()))
lc.set_array(values)
lc.set_linewidths(15 * values)
ax.add_collection(lc)

# Add a label to each node. The challenge here is that we want to
# position the labels to avoid overlap with other labels
for index, (name, label, (x, y)) in enumerate(
        zip(names, labels, embedding.T)):

    dx = x - embedding[0]
    dx[index] = 1
    dy = y - embedding[1]
    dy[index] = 1
    this_dx = dx[np.argmin(np.abs(dy))]
    this_dy = dy[np.argmin(np.abs(dx))]
    if this_dx > 0:
        horizontalalignment = 'left'
        x = x + .002
    else:
        horizontalalignment = 'right'
        x = x - .002
    if this_dy > 0:
        verticalalignment = 'bottom'
        y = y + .002
    else:
        verticalalignment = 'top'
        y = y - .002
    plt.text(x, y, name, size=10,
             horizontalalignment=horizontalalignment,
             verticalalignment=verticalalignment,
             bbox=dict(facecolor='w',
                       edgecolor=plt.cm.spectral(label / float(n_labels)),
                       alpha=.6))

plt.xlim(embedding[0].min() - .15 * embedding[0].ptp(),
         embedding[0].max() + .10 * embedding[0].ptp(),)
plt.ylim(embedding[1].min() - .03 * embedding[1].ptp(),
         embedding[1].max() + .03 * embedding[1].ptp())

plt.show()